<a href="https://colab.research.google.com/github/ismetoksum/BYTE-Buyuculeri-Turkish-Sentiment-Analysis-Acikhack2024TDDI/blob/main/TeknofestDdi2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
pip install emot

In [ ]:
import re
import sys
from nltk.stem.porter import PorterStemmer
from emot.emo_unicode import UNICODE_EMOJI
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist

In [ ]:
import matplotlib
from sklearn.decomposition import PCA
from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier

import json
import pandas as pd
import random
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import f1_score, classification_report
from spacy.tokenizer import Tokenizer
from spacy.lang.tr import Turkish
import numpy as np
import warnings
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch

In [ ]:
df = pd.read_csv("/content/Clash Of Clans Yorumları.csv",encoding='unicode_escape',on_bad_lines='skip')
df

,Column1
0,Oyun güzel. Kötü tarafý baðýmlýlýk yapýyor. En...
1,Supercell hesabima giriþ yapamýyorum üstelik b...
2,Oyun gayet güzel bir sonraki güncellemede süpe...
3,Bu oyunu seviyorum ama yeni köy dasarladýðýmda...
4,Supercell hesabima giriþ yapamýyorum üstelik b...
...,...
174,Oyun hayvan gibi internet istiyo savaþlarda ba...
175,Bence oyun güzel ama global kesinlikle getiril...
176,Oyunun aktifliðini ve keyfini arttýrmak için o...
177,Merhaba clash of clans size bildirmek istediði...


In [ ]:
df = pd.read_csv("/content/Clash Of Clans Yorumları.csv",encoding='unicode_escape',on_bad_lines='skip')
df

,Column1
0,Oyun güzel. Kötü tarafý baðýmlýlýk yapýyor. En...
1,Supercell hesabima giriþ yapamýyorum üstelik b...
2,Oyun gayet güzel bir sonraki güncellemede süpe...
3,Bu oyunu seviyorum ama yeni köy dasarladýðýmda...
4,Supercell hesabima giriþ yapamýyorum üstelik b...
...,...
174,Oyun hayvan gibi internet istiyo savaþlarda ba...
175,Bence oyun güzel ama global kesinlikle getiril...
176,Oyunun aktifliðini ve keyfini arttýrmak için o...
177,Merhaba clash of clans size bildirmek istediði...


In [ ]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("emre/turkish-sentiment-analysis", use_auth_token=True)

tokenizer = AutoTokenizer.from_pretrained("emre/turkish-sentiment-analysis", use_auth_token=True)

inputs = tokenizer("senden nefret ediyorum", return_tensors="pt")

outputs = model(**inputs)
print(outputs)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


SequenceClassifierOutput(loss=None, logits=tensor([[ 4.3386, -4.8180, -0.7359]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)


In [ ]:
from transformers import pipeline
import pandas as pd

classifier = pipeline("text-classification", model="emre/turkish-sentiment-analysis", device=0)  # Use GPU, or -1 for CPU

# Assuming the text is in the first column of the DataFrame
texts = df.iloc[:, 0].tolist()
results = classifier(texts)

print(results)

[{'label': 'Positive', 'score': 0.9506049752235413}, {'label': 'Negative', 'score': 0.9937147498130798}, {'label': 'Positive', 'score': 0.9873393774032593}, {'label': 'Negative', 'score': 0.9579712152481079}, {'label': 'Negative', 'score': 0.9937147498130798}, {'label': 'Positive', 'score': 0.9947667121887207}, {'label': 'Positive', 'score': 0.9673683047294617}, {'label': 'Positive', 'score': 0.9562215805053711}, {'label': 'Negative', 'score': 0.9535788297653198}, {'label': 'Positive', 'score': 0.9805650115013123}, {'label': 'Positive', 'score': 0.9899933934211731}, {'label': 'Positive', 'score': 0.9790135622024536}, {'label': 'Positive', 'score': 0.9508619904518127}, {'label': 'Positive', 'score': 0.8624043464660645}, {'label': 'Negative', 'score': 0.7023057341575623}, {'label': 'Negative', 'score': 0.9969713687896729}, {'label': 'Positive', 'score': 0.9918811917304993}, {'label': 'Negative', 'score': 0.9860678315162659}, {'label': 'Negative', 'score': 0.9845501780509949}, {'label': '

In [ ]:
def FunctionBERTSentiment(inpText):
  return(classifier(inpText)[0]['label'])

# Calling the function
FunctionBERTSentiment(inpText="çok kötü olmuş")

'Negative'

In [ ]:
# FunctionBERTSentiment fonksiyonunu tanımlayın (eğer henüz tanımlanmamışsa)
def FunctionBERTSentiment(text):
    result = classifier(text)
    return result[0]['label']  # veya istediğiniz başka bir değer

# Metin verisinin bulunduğu sütunu belirtin (örneğin 'Tweet' sütunu)
df['Sentiment'] = df['Column1'].apply(FunctionBERTSentiment)
df.head(10)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


,Column1,Sentiment
0,Oyun güzel. Kötü tarafý baðýmlýlýk yapýyor. En...,Positive
1,Supercell hesabima giriþ yapamýyorum üstelik b...,Negative
2,Oyun gayet güzel bir sonraki güncellemede süpe...,Positive
3,Bu oyunu seviyorum ama yeni köy dasarladýðýmda...,Negative
4,Supercell hesabima giriþ yapamýyorum üstelik b...,Negative
5,Açýkçasý mükemmel bir oyun diðer supercell oyu...,Positive
6,Oyun cidden çok güzel klan savaþlarýnda eþleþm...,Positive
7,Oyun güzel ama þöyle eksikleri var yükseltme y...,Positive
8,oyun iyi bir oyun oynalýnýr ama bir kaç eksikl...,Negative
9,Gerçekten iyi bir oyun ama 1 inþaat yükseltmes...,Positive


In [ ]:
!pip install fastapi uvicorn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 6.6 MB/s eta 0:00:00


In [ ]:
from fastapi import FastAPI
from pydantic import BaseModel
from transformers import pipeline

app = FastAPI(title="Türkçe Duygu Analizi API", description="BERT tabanlı Türkçe duygu analizi API'si")

# Sentiment analiz modelini yükleyin
classifier = pipeline("text-classification", model="emre/turkish-sentiment-analysis")

class SentimentRequest(BaseModel):
    text: str

class SentimentResponse(BaseModel):
    sentiment: str
    score: float

@app.post("/predict", response_model=SentimentResponse)
def predict_sentiment(request: SentimentRequest):
    result = classifier(request.text)[0]
    return SentimentResponse(sentiment=result['label'], score=result['score'])

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
!pip install pyngrok

In [ ]:
!pip install fastapi uvicorn

In [ ]:
!ngrok authtoken 2j4ohqLlsZ99rhNva7yIG2Uji0t_5fXU52P9j4XEHzFT9jHZV

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

# Authtoken'ı ayarlayın (sadece bir kez yapmanız yeterli)
!ngrok authtoken 2j4ohqLlsZ99rhNva7yIG2Uji0t_5fXU52P9j4XEHzFT9jHZV

nest_asyncio.apply()

# Ngrok tüneli oluşturun
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

# Uvicorn ile uygulamayı çalıştırın
uvicorn.run(app, port=8000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: https://9123-34-124-202-148.ngrok-free.app


INFO:     Started server process [224]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2a02:4e0:2d44:37e9:7cd4:4f58:22ff:33e:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2a02:4e0:2d44:37e9:7cd4:4f58:22ff:33e:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     2a02:4e0:2d44:37e9:7cd4:4f58:22ff:33e:0 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     2a02:4e0:2d44:37e9:7cd4:4f58:22ff:33e:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2a02:4e0:2d44:37e9:7cd4:4f58:22ff:33e:0 - "GET / HTTP/1.1" 404 Not Found


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn

nest_asyncio.apply()

# Ngrok tüneli oluşturun
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)

# Uvicorn ile uygulamayı çalıştırın
uvicorn.run(app, port=8000)

INFO:     Started server process [224]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://6933-34-124-202-148.ngrok-free.app
INFO:     2a02:4e0:2d44:37e9:7cd4:4f58:22ff:33e:0 - "GET / HTTP/1.1" 404 Not Found
INFO:     2a02:4e0:2d44:37e9:7cd4:4f58:22ff:33e:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [224]
